## Import

In [ ]:
import os
import yaml
import warnings
from typing import List
from tiktoken import encoding_for_model

from utils import get_openai_api_key
from utils_data import add_slot, is_slot_conflict, remove_slot, access_timetable, access_timetable_with_date

import crewai
from crewai import Agent, Task, Crew, Process
from pydantic import BaseModel, Field
from crewai.tools import BaseTool
from crewai.flow import Flow, router, listen, start
from crewai import LLM


warnings.filterwarnings('ignore')

In [2]:
print(os.environ["OPENAI_MODEL_NAME"])

gpt-4o-mini


In [3]:
#TIMETABLE = {}

"""{'06/10/2025': [{'weekday': 'Monday',
   'startTime': '17:00',
   'endTime': '18:00',
   'title': 'Meeting with Boss'}],
 '13/10/2025': [{'weekday': 'Monday',
   'startTime': '17:00',
   'endTime': '18:00',
   'title': 'Meeting with Boss'}]}"""

"""
    "date": [{"startTime", "endTime", "title"}, {...}, {...}, ...],
    "anotherdate": [...], 
"""

'\n    "date": [{"startTime", "endTime", "title"}, {...}, {...}, ...],\n    "anotherdate": [...], \n'

#### Get api key

In [4]:
openai_api_key = get_openai_api_key()

## Setup configs

In [5]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Pydantic models

In [6]:
class EventSlot(BaseModel):
    date: str = Field(..., description="The date of the event. The format is: DD/MM/YYYY")
    weekday: str = Field(..., description="The date's week day.")
    startTime: str = Field(..., description="Start time of the slot. The format is: HH:MM")
    endTime: str = Field(..., description="End time of the slot. The format is: HH:MM")
    title: str = Field(..., description="Short title to summarize the event's essential information.")

class MultipleSlots(BaseModel):
    slots: List[EventSlot] = Field(..., description="List of event slots")

## Tools

In [7]:
from datetime import date
from typing import Type


from datetime import datetime

class InputDate(BaseModel):
    date: str = Field(..., description="The date of the event. The format is: DD/MM/YYYY")
    weekday: str = Field(..., description="The date's week day.")

class ValidDateTool(BaseTool):
    name: str = "Valid Date Tool"
    description: str = "Tool that checks if the date and week day are valid"
    args_schema: Type[BaseModel] = InputDate

    def _run(self, date: str, weekday: str):
        try:
            datetime.strptime(date, "%d/%m/%Y")
        except ValueError:
            return f"Error: {str(e)}. Please provide a valid date."
        
        try:
            date_obj = datetime.strptime(date, "%d/%m/%Y")
            correct_weekday = date_obj.strftime("%A")  # Full name (e.g., "Monday")
            correct_weekday.lower().startswith(weekday.lower())
            return "Valid date"
        except ValueError as e:
            return f"Error: {str(e)}. The week day does not corresponds to the date"
        
class AddSlot(BaseTool):
    name: str = "Add Slot Tool"
    description: str = "Tool that adds slots to the timetable using these informations: date, weekday, start time, end time and the title of the event."
    args_schema: Type[BaseModel] = MultipleSlots

    def _run(self, slots: MultipleSlots):
        for eventSlot in slots:
            res = add_slot(eventSlot)
            if res != "SUCCESS":
                return res
        return "Successfully added slot(s)"
    
class RemoveSlot(BaseTool):
    name: str = "Remove Slot Tool"
    description: str = "Tool that removes slots to the timetable using these informations: date, weekday, start time, end time and the title of the event.\n" \
    "Do not call this tool again if already successful."
    args_schema: Type[BaseModel] = MultipleSlots

    def _run(self, slots: MultipleSlots):
        res = ""
        for eventSlot in slots:
            res += remove_slot(eventSlot) + "\n"
        return res

class AccessTimetableWithDate(BaseTool):
    name: str = "Access Timetable"
    description: str = "Tool that enables access to slots on a specific date in dict (format of date DD/MM/YYYY)"
    
    def _run(self, date_input: str):
        return access_timetable_with_date(date_input)
            
    
class CheckConflict(BaseTool):
    name: str = "Check Conflict Tool"
    description: str = "Tool that checks if there is a conflict between an event and an already setup event in the timetable. " \
    "Returns the list of events where there is a conflict with already setup events in the timetable."
    args_schema: Type[BaseModel] = MultipleSlots

    def _run(self, slots: MultipleSlots):
        conflicts = []
        for slot in slots:
            res = is_slot_conflict(slot)
            if res == "There is a conflict":
                conflicts.append(slot)
        return conflicts


## Agents

In [8]:
converter = Agent(
    config=agents_config["converter"],
    inject_date=True,
    date_format="%A, %B %d, %Y",
    tools=[ValidDateTool()]
)

assistant = Agent(
    config=agents_config["assistant"],
    tools=[AddSlot(), CheckConflict(), RemoveSlot(), AccessTimetableWithDate()],
    inject_date=True
)

informer = Agent(
    config=agents_config["informer"],
    tools=[AccessTimetableWithDate()]
)

manager = Agent(
    config=agents_config["manager"]
)

## Tasks

In [9]:
conversion = Task(
    config=tasks_config["conversion"],
    agent=converter,
    output_pydantic=MultipleSlots
)

manage_timetable = Task(
    config=tasks_config["manage_timetable"],
    agent=assistant
)

answer_question = Task(
    config=tasks_config["answer_question"],
    agent=informer
)

## Crew

In [10]:
crew1 = Crew(
    agents=[converter, assistant],
    tasks=[conversion, manage_timetable],
    verbose=True
)

crew2 = Crew(
    agents=[informer],
    tasks=[answer_question],
    verbose=True
)

# Flow

In [26]:
class MyFlow(Flow):
    @start()
    def analyze_input(self):
        self.total_token_usage = 0
        return self.state['user_input']
    
    @router(analyze_input)
    def route_based_on_content(self, user_input):
        llm = LLM(model="openai/gpt-4o-mini")
        prompt =  f"""
        Analyze this input and determine which agent should handle it:
        - Return "Timetable Manager" for request asking to add or remove an event, for example:
                - Can you add this event
                - I have this event today
                - Can you remove this event from the timetable
        - Return "Timetable Informer" for questions about the timetable, for example:
                - What events are scheduled on a given date.
                - When a specific event is happening.
                - Which days are free or busy.
        
        Input: {user_input}
        
        Return only the agent name.
        """

        enc = encoding_for_model("gpt-4o-mini")
        num_tokens = len(enc.encode(prompt))

        result = llm.call(prompt).strip()
        
        output_tokens = len(enc.encode(result))
        self.total_token_usage = num_tokens + output_tokens
        
        print(result)

        return "Timetable Manager" if result == "Timetable Manager" else "Timetable Informer"

    @listen("Timetable Manager")
    def call_timetable_manager(self):
        result = crew1.kickoff(inputs={"event": self.state['user_input']})
        self.total_token_usage += result.token_usage.total_tokens
        return result
    
    @listen("Timetable Informer")
    def call_timetable_informer(self):
        result = crew2.kickoff(inputs={"event": self.state['user_input']})
        self.total_token_usage += result.token_usage.total_tokens
        return result
    


## Execution

In [13]:
flow = MyFlow()
result = await flow.kickoff_async(inputs={"user_input": "Can you describe me the events on the 07/10/2025 ?"})

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: MyFlow                                                                                                   │
│  ID: 7175ae5e-d5da-4c98-b1ec-2dd5024e71fd                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 Flow started with ID: 7175ae5e-d5da-4c98-b1ec-2dd5024e71fd

Timetable Informer

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0b745eb2-fbe5-406b-b9f4-e6ffd627083c                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Timetable information expert                                                                            │
│                                                                                                                 │
│  Task: Answer the following question about the timetable at your disposal: Can you describe me the events on    │
│  the 07/10/2025 ? You can access it through the tools. Only you can access it. The user might ask questions     │
│  such as:                                                                                                       │
│    - What events are scheduled on a given date.                                                                 │
│    - When a specific event is happening.                                                                        │
│    - Which days are free or busy.                                                                               │
│  Make sure your answers are accurate and based on the timetable data.                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  [{'weekday': 'Tuesday', 'startTime': '19:30', 'endTime': '21:30', 'title': 'Pizzeria Popolare'}, {'weekday':   │
│  'Tuesday', 'startTime': '08:00', 'endTime': '09:00', 'title': 'Gym Session'}, {'weekday': 'Tuesday',           │
│  'startTime': '17:30', 'endTime': '18:30', 'title': 'Job Interview with Margo'}, {'weekday': 'Tuesday',         │
│  'startTime': '13:00', 'endTime': '14:00', 'title': 'Haircut'}]                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Timetable information expert                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  On 07/10/2025 (Tuesday), the following events are scheduled:                                                   │
│  1. Gym Session from 08:00 to 09:00                                                                             │
│  2. Haircut from 13:00 to 14:00                                                                                 │
│  3. Job Interview with Margo from 17:30 to 18:30                                                                │
│  4. Pizzeria Popolare from 19:30 to 21:30                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 01b248ec-d421-4f57-849c-8a493536d192                                                                     │
│  Agent: Timetable information expert                                                                            │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0b745eb2-fbe5-406b-b9f4-e6ffd627083c                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: On 07/10/2025 (Tuesday), the following events are scheduled:                                     │
│  1. Gym Session from 08:00 to 09:00                                                                             │
│  2. Haircut from 13:00 to 14:00                                                                                 │
│  3. Job Interview with Margo from 17:30 to 18:30                                                                │
│  4. Pizzeria Popolare from 19:30 to 21:30                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Flow Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name: MyFlow                                                                                                   │
│  ID: 7175ae5e-d5da-4c98-b1ec-2dd5024e71fd                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [14]:
from IPython.display import Markdown
Markdown(result.raw)

On 07/10/2025 (Tuesday), the following events are scheduled:  
1. Gym Session from 08:00 to 09:00  
2. Haircut from 13:00 to 14:00  
3. Job Interview with Margo from 17:30 to 18:30  
4. Pizzeria Popolare from 19:30 to 21:30

In [15]:
#inputs_array = [{"event": "Add a gym session on the 10th october 8am-9am"}
#                """{"event": "Can you delete the meeting on the 6th october."},
#                {"event": "Camille's birthday is on the 15/05/2025."}"""]

In [16]:
#result = crew.kickoff_for_each(inputs=inputs_array)

## Cost

In [17]:
import pandas as pd

costs = 0.150 * (flow.total_token_usage) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
#df_usage_metrics = pd.DataFrame([flow.usage_metrics.dict()])
#df_usage_metrics

Total costs: $0.0002


In [ ]:
class MyFlow(Flow):
    @start()
    def analyze_input(self):
        self.total_token_usage = 0
        return self.state['user_input']
    
    @router(analyze_input)
    def route_based_on_content(self, user_input):
        # llm = LLM(model="openai/gpt-4o-mini")
        # prompt =  f"""
        # Analyze this input and determine which agent should handle it:
        # - Return "Timetable Manager" for request asking to add or remove an event, for example:
        #         - Can you add this event
        #         - I have this event today
        #         - Can you remove this event from the timetable
        # - Return "Timetable Informer" for questions about the timetable, for example:
        #         - What events are scheduled on a given date.
        #         - When a specific event is happening.
        #         - Which days are free or busy.
        
        # Input: {user_input}
        
        # Return only the agent name.
        # """

        # enc = encoding_for_model("gpt-4o-mini")
        # num_tokens = len(enc.encode(prompt))

        # result = llm.call(prompt).strip()

        return "Timetable Manager"
        
        # output_tokens = len(enc.encode(result))
        # self.total_token_usage = num_tokens + output_tokens
        
        # print(result)

        # return "Timetable Manager" if result == "Timetable Manager" else "Timetable Informer"

    @listen("Timetable Manager")
    def call_timetable_manager(self):
        result = crew1.kickoff(inputs={"event": self.state['user_input']})
        self.total_token_usage += result.token_usage.total_tokens
        return result
    
    @listen("Timetable Informer")
    def call_timetable_informer(self):
        result = crew2.kickoff(inputs={"event": self.state['user_input']})
        self.total_token_usage += result.token_usage.total_tokens
        return result
    


In [27]:
flow = MyFlow()

flow.plot()

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: MyFlow                                                                                                   │
│  ID: dd383749-fe6f-4103-96a9-e49896897a94                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Plot saved as crewai_flow.html


In [25]:
import webbrowser

webbrowser.open("crewai_flow.html")


True